# Minimize Net System Carbon Emission with Even_flow Constraints

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Install Pandas Library Version 2.0.3
%pip install pandas==2.0.3

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Install libcbm Module from GitHub and Import it
install_libcbm = False
if install_libcbm:
    %pip install -U git+https://github.com/cat-cfs/libcbm_py.git@main
    import libcbm
    libcbm.__path__

In [4]:
# Import Modules
import matplotlib.pyplot as plt
import pandas as pd
import ws3.forest, ws3.core
import math
import libcbm
import os

In [5]:
from util import compile_scenario, plot_scenario, plugin_c_curves, cmp_c_z, cmp_c_caa, cmp_c_ci

In [6]:
#Install Module
%pip install gurobipy
#Import Module
import gurobipy as grb

Note: you may need to restart the kernel to use updated packages.


In [7]:
#Define constants from product carbon estimation
# Define the product half-lives（>0.69)
half_life_paper = 2
# Define the allocation distribution
proportion_solid_wood = 1.0
# Define the untilization rate
util = 0.85
# Define the HWPs displacement factor 
displacement_factor_solid = 2.2
# Define the Credibility
credit =  1.0

In [8]:
# Ensure the results directories exist
base_dir = f'results/minimize_emission_loop/half_life/even_flow/df_{displacement_factor_solid}'
harvest_dir = os.path.join(base_dir, 'harvest')
stock_dir = os.path.join(base_dir, 'stock')
emission_dir = os.path.join(base_dir, 'emission')

In [9]:
iteration = 200

In [10]:
# Define Disturance Types
disturbance_type_mapping = [{'user_dist_type': 'harvest', 'default_dist_type': 'Clearcut harvesting without salvage'},
                            {'user_dist_type': 'fire', 'default_dist_type': 'Wildfire'}]

In [11]:
# Define Carbon Pools
biomass_pools = ['SoftwoodMerch','SoftwoodFoliage', 'SoftwoodOther', 'SoftwoodCoarseRoots','SoftwoodFineRoots',                        
                 'HardwoodMerch', 'HardwoodFoliage', 'HardwoodOther', 'HardwoodCoarseRoots', 'HardwoodFineRoots']
dom_pools = ['AboveGroundVeryFastSoil', 'BelowGroundVeryFastSoil', 'AboveGroundFastSoil', 'BelowGroundFastSoil',
             'MediumSoil', 'AboveGroundSlowSoil', 'BelowGroundSlowSoil', 'SoftwoodStemSnag', 'SoftwoodBranchSnag',
             'HardwoodStemSnag', 'HardwoodBranchSnag']
emissions_pools = ['CO2', 'CH4', 'CO', 'NO2']
products_pools = ['Products']
ecosystem_pools = biomass_pools + dom_pools
all_pools = biomass_pools + dom_pools + emissions_pools + products_pools

# Define Carbon Fluxes
annual_process_fluxes = [
    'DecayDOMCO2Emission',
    'DeltaBiomass_AG',
    'DeltaBiomass_BG',
    'TurnoverMerchLitterInput',
    'TurnoverFolLitterInput',
    'TurnoverOthLitterInput',
    'TurnoverCoarseLitterInput',
    'TurnoverFineLitterInput',
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir'
]

npp_fluxes=[
    'DeltaBiomass_AG', 
    'DeltaBiomass_BG'
]
decay_emissions_fluxes = [
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir'
]

disturbance_production_fluxes = [
    'DisturbanceSoftProduction',
    'DisturbanceHardProduction',
    'DisturbanceDOMProduction'   
]

disturbance_emissions_fluxes = [
    'DisturbanceMerchToAir',
    'DisturbanceFolToAir',
    'DisturbanceOthToAir',
    'DisturbanceCoarseToAir',
    'DisturbanceFineToAir',
    'DisturbanceVFastAGToAir',
    'DisturbanceVFastBGToAir',
    'DisturbanceFastAGToAir',
    'DisturbanceFastBGToAir',
    'DisturbanceMediumToAir',
    'DisturbanceSlowAGToAir',
    'DisturbanceSlowBGToAir',
    'DisturbanceSWStemSnagToAir',
    'DisturbanceSWBranchSnagToAir',
    'DisturbanceHWStemSnagToAir',
    'DisturbanceHWBranchSnagToAir'   
]

all_fluxes = [
    'DisturbanceCO2Production',
    'DisturbanceCH4Production',
    'DisturbanceCOProduction',
    'DisturbanceBioCO2Emission',
    'DisturbanceBioCH4Emission',
    'DisturbanceBioCOEmission',
    'DecayDOMCO2Emission',
    'DisturbanceSoftProduction',
    'DisturbanceHardProduction',
    'DisturbanceDOMProduction',
    'DeltaBiomass_AG',
    'DeltaBiomass_BG',
    'TurnoverMerchLitterInput',
    'TurnoverFolLitterInput',
    'TurnoverOthLitterInput',
    'TurnoverCoarseLitterInput',
    'TurnoverFineLitterInput',
    'DecayVFastAGToAir',
    'DecayVFastBGToAir',
    'DecayFastAGToAir',
    'DecayFastBGToAir',
    'DecayMediumToAir',
    'DecaySlowAGToAir',
    'DecaySlowBGToAir',
    'DecaySWStemSnagToAir',
    'DecaySWBranchSnagToAir',
    'DecayHWStemSnagToAir',
    'DecayHWBranchSnagToAir',
    'DisturbanceMerchToAir',
    'DisturbanceFolToAir',
    'DisturbanceOthToAir',
    'DisturbanceCoarseToAir',
    'DisturbanceFineToAir',
    'DisturbanceDOMCO2Emission',
    'DisturbanceDOMCH4Emission',
    'DisturbanceDOMCOEmission',
    'DisturbanceMerchLitterInput',
    'DisturbanceFolLitterInput',
    'DisturbanceOthLitterInput',
    'DisturbanceCoarseLitterInput',
    'DisturbanceFineLitterInput',
    'DisturbanceVFastAGToAir',
    'DisturbanceVFastBGToAir',
    'DisturbanceFastAGToAir',
    'DisturbanceFastBGToAir',
    'DisturbanceMediumToAir',
    'DisturbanceSlowAGToAir',
    'DisturbanceSlowBGToAir',
    'DisturbanceSWStemSnagToAir',
    'DisturbanceSWBranchSnagToAir',
    'DisturbanceHWStemSnagToAir',
    'DisturbanceHWBranchSnagToAir'
]

grossgrowth_ag = [
    "DeltaBiomass_AG",
    "TurnoverMerchLitterInput",
    "TurnoverFolLitterInput",
    "TurnoverOthLitterInput",
]

grossgrowth_bg = [
    "DeltaBiomass_BG",
    "TurnoverCoarseLitterInput",
    "TurnoverFineLitterInput",
]

product_flux = [
     "DisturbanceSoftProduction",
     "DisturbanceHardProduction",
     "DisturbanceDOMProduction",
]

In [12]:
# Define Sum Carbon Pools and Sum Carbon Fluxes
total_emission = decay_emissions_fluxes + disturbance_emissions_fluxes
gross_growth = grossgrowth_ag + grossgrowth_bg

sum_pools = ['ecosystem', 'biomass', 'DOM']
sum_fluxes = ['total_emission', 'gross_growth', 'net_emission']

In [13]:
def gen_scenario(fm, name='base', util=1.0, harvest_acode='harvest',
                 cflw_ha={}, cflw_hv={}, 
                 cgen_ha={}, cgen_hv={}, 
                 cgen_gs={}, tvy_name='totvol', cp_name='ecosystem', obj_mode='min_hv', mask=None):
    
    from functools import partial
    import numpy as np
    coeff_funcs = {}
    cflw_e = {}
    cgen_data = {}
    acodes = ['null', harvest_acode] # define list of action codes
    vexpr = '%s * %0.2f' % (tvy_name, util) # define volume expression

    #Define constants from product carbon estimation

    if obj_mode == 'max_hv': # maximize harvest volume
        sense = ws3.opt.SENSE_MAXIMIZE 
        zexpr = vexpr
    elif obj_mode == 'min_hv': # maximize harvest volume
        sense = ws3.opt.SENSE_MINIMIZE 
        zexpr = vexpr
    else:
        raise ValueError('Invalid obj_mode: %s' % obj_mode)
    
    coeff_funcs['z'] = partial(cmp_c_se, expr=vexpr, yname=cp_name) # define objective function coefficient function for net system carbon emission
    
    T = fm.periods
    if cflw_ha: # define even flow constraint (on harvest area)
        cname = 'cflw_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_ha
    if cflw_hv: # define even flow constraint (on harvest volume)
        cname = 'cflw_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cflw_e[cname] = cflw_hv         
    if cgen_ha: # define general constraint (harvest area)
        cname = 'cgen_ha'
        coeff_funcs[cname] = partial(cmp_c_caa, expr='1.', acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_ha
    if cgen_hv: # define general constraint (harvest volume)
        cname = 'cgen_hv'
        coeff_funcs[cname] = partial(cmp_c_caa, expr=vexpr, acodes=[harvest_acode], mask=None) 
        cgen_data[cname] = cgen_hv
    if cgen_gs: # define general constraint (growing stock)
        cname = 'cgen_gs'
        coeff_funcs[cname] = partial(cmp_c_ci, yname=tvy_name, mask=None)
        cgen_data[cname] = cgen_gs

    return fm.add_problem(name, coeff_funcs, cflw_e, cgen_data=cgen_data, acodes=acodes, sense=sense, mask=mask)

In [14]:
def run_scenario(fm, scenario_name='base'):
    #Import Module
    import gurobipy as grb
    
    cflw_ha = {}
    cflw_hv = {}
    cgen_ha = {}
    cgen_hv = {}
    cgen_gs = {}
    
    # define harvest area and harvest volume even-flow constraints
    cflw_ha = ({p:0.05 for p in fm.periods}, 1)
    cflw_hv = ({p:0.05 for p in fm.periods}, 1)

    if scenario_name == 'base': 
        # Base scenario
        print('running base scenario')
    else:
        assert False # bad scenario name
      
    p = gen_scenario(fm=fm, 
                     name=scenario_name, 
                     cflw_ha=cflw_ha, 
                     cflw_hv=cflw_hv,
                     cgen_ha=cgen_ha,
                     cgen_hv=cgen_hv,
                     cgen_gs=cgen_gs,)

    fm.reset()
    m = p.solve()

    if m.status != grb.GRB.OPTIMAL:
        print('Model not optimal.')
        # sys.exit()
        
    sch = fm.compile_schedule(p)
    fm.apply_schedule(sch, 
                      force_integral_area=False, 
                      override_operability=False,
                      fuzzy_age=False,
                      recourse_enabled=False,
                      verbose=False,
                      compile_c_ycomps=True)
    return p

In [15]:
# Function to initialize the ForestModel with Carbon Yield Curves
def initialize_forest_model():
    base_year = 2020
    horizon = 20
    period_length = 10
    max_age = 1000
    tvy_name = 'totvol'

    fm = ws3.forest.ForestModel(model_name='tsa24',
                                model_path='data/woodstock_model_files_tsa24',
                                base_year=base_year,
                                horizon=horizon,
                                period_length=period_length,
                                max_age=max_age)

    fm.import_landscape_section()
    fm.import_areas_section()
    fm.import_yields_section()
    fm.import_actions_section()
    fm.import_transitions_section()
    fm.initialize_areas()
    fm.add_null_action()
    fm.reset_actions()
    fm.grow()
    
    # Flag 'harvest' as a harvesting action in the ws3 model
    harvest_acode = 'harvest'
    fm.actions[harvest_acode].is_harvest = True

    # Plug-in Carbon Yield Curves
    c_curves_p = pd.read_pickle("curves/c_curves_p_700_age1_new.pkl")
    c_curves_f = pd.read_pickle("curves/c_curves_f_700_age1_new.pkl")
    plugin_c_curves(fm, c_curves_p, c_curves_f, sum_pools, sum_fluxes)
    
    return fm

In [16]:
def track_system_stock(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util):
    
    hwps_solid_pool = 0.
    hwps_paper_pool = 0.
    hwps_residue_pool = 0.

    sys_pool_list = []
    eco_pool_list = []
    hwps_solid_pool_list = []
    hwps_paper_pool_list = []
    hwps_residue_pool_list = []
    hwps_sum_pool_list = []
    
    # Calculate decay rates based on half-lives
    k_solid_wood = math.log(2) / half_life_solid_wood
    k_paper = math.log(2) / half_life_paper

    # Define the allocation distribution
    proportion_paper = 1 - proportion_solid_wood

    # Constants
    wood_density = 460 #(Kennedy, 1965)
    carbon_content = 0.5

    for period in fm.periods:

        # Track Ecosystem Carbon Stock
        eco_pool = fm.inventory(period, 'ecosystem')
        
        # Calculate new product stocks
        new_product_stock = fm.compile_product(period, 'totvol', acode='harvest') * wood_density * carbon_content / 1000 # Convert kg to ton
        # Calculate product stock
        hwps_solid_pool = hwps_solid_pool * (1 - k_solid_wood)**10 + new_product_stock * util * proportion_solid_wood
        hwps_paper_pool = hwps_paper_pool * (1 - k_paper)**10 + new_product_stock * util * proportion_paper
        hwps_residue_pool = new_product_stock * (1.0 - util)
        hwps_sum_pool = hwps_solid_pool + hwps_paper_pool

        # Calculate total system carbon stock
        sys_pool = eco_pool + hwps_sum_pool

        # Update stock lists for this period
        sys_pool_list.append(sys_pool)
        eco_pool_list.append(eco_pool)
        hwps_solid_pool_list.append(hwps_solid_pool)
        hwps_paper_pool_list.append(hwps_paper_pool)
        hwps_residue_pool_list.append(hwps_residue_pool)
        hwps_sum_pool_list.append(hwps_sum_pool)

    # Prepare data for plotting
    data = {
        'period': fm.periods,
        'solid_wood': hwps_solid_pool_list,
        'paper': hwps_paper_pool_list,
        'residue': hwps_residue_pool_list,
        'sum_product': hwps_sum_pool_list,
        'ecosystem': eco_pool_list,
        'system': sys_pool_list
    }

    df = pd.DataFrame(data)

    return df

In [17]:
def track_system_emission(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util, displacement_factor_solid, credit):
    
    hwps_solid_pool = 0.
    hwps_paper_pool = 0.
    hwps_residue_pool = 0.
    hwps_solid_emission = 0.
    hwps_paper_emission = 0.
    hwps_residue_emission = 0.

    sys_emission_list = []
    eco_emission_list = []
    hwps_solid_emission_list = []
    hwps_paper_emission_list = []
    hwps_residue_emission_list = []
    hwps_sum_emission_list = []
    
    # Calculate decay rates based on half-lives
    k_solid_wood = math.log(2) / half_life_solid_wood
    k_paper = math.log(2) / half_life_paper

    # Define the allocation distribution
    proportion_paper = 1-proportion_solid_wood

    # Constants
    wood_density = 460 #(Kennedy, 1965)
    carbon_content = 0.5

    for period in fm.periods:

        # Calculate ecosytem emission
        eco_emission = fm.inventory(period, 'net_emission')

        # Calculate new product stocks
        new_product_stock = fm.compile_product(period, 'totvol', acode='harvest') * wood_density * carbon_content / 1000

        # Calculate total product stock
        hwps_solid_pool = hwps_solid_pool * (1 - k_solid_wood)**10 + new_product_stock * util * proportion_solid_wood
        hwps_paper_pool = hwps_paper_pool * (1 - k_paper)**10 + new_product_stock * util * proportion_paper
        hwps_residue_pool =  new_product_stock * (1- util)

        # Calculate product emission
        hwps_solid_emission = hwps_solid_pool*(1- (1 - k_solid_wood)**10) * 44 / 12 #Convert from C to CO2
        hwps_paper_emission = hwps_paper_pool*(1- (1 - k_paper)**10) * 44 / 12 #Convert from C to CO2
        hwps_residue_emission = hwps_residue_pool * 44 / 12 #Convert from C to CO2
        
        hwps_sum_emission = hwps_solid_emission + hwps_paper_emission + hwps_residue_emission

        # Calculate Substitution Effect
        substitution_effect_solid = new_product_stock * util * proportion_solid_wood * displacement_factor_solid * credit # Emissions avoided by using HWPs
        substitution_effect_paper = 0.
        substitution_effect_residue = 0.
        substitution_effect_sum = (substitution_effect_solid + substitution_effect_paper + substitution_effect_residue) * 44 / 12 #Convert from C to CO2
        
        # Calculate net system carbon emission
        sys_emission = (eco_emission + hwps_sum_emission - substitution_effect_sum)

        # Update stock lists for this period
        sys_emission_list.append(sys_emission)
        eco_emission_list.append(eco_emission)
        hwps_solid_emission_list.append(hwps_solid_emission)
        hwps_paper_emission_list.append(hwps_paper_emission)
        hwps_residue_emission_list.append(hwps_residue_emission)
        hwps_sum_emission_list.append(hwps_sum_emission)

    # Prepare data for plotting
    data = {
        'period': fm.periods,
        'solid_wood': hwps_solid_emission_list,
        'paper': hwps_paper_emission_list,
        'residue': hwps_residue_emission_list,
        'sum_product': hwps_sum_emission_list,
        'ecosystem': eco_emission_list,
        'system': sys_emission_list
    }

    df = pd.DataFrame(data)

    return df

In [18]:
def run_scenario_and_save(fm, half_life_solid_wood):    
    # Maximize Carbon Stock
    print(f'half_life_solid_wood = {half_life_solid_wood}')
    fm.add_null_action()
    p = run_scenario(fm, 'base')

    # Extract results
    df_harvest = compile_scenario(fm)
    df_stock = track_system_stock(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util)
    df_emission = track_system_emission(fm, half_life_solid_wood, half_life_paper, proportion_solid_wood, util, displacement_factor_solid, credit)
    
    # Save results with filenames using half_life_solid_wood
    df_harvest.to_csv(os.path.join(harvest_dir, f'{half_life_solid_wood}.csv'), index=False)
    df_stock.to_csv(os.path.join(stock_dir, f'{half_life_solid_wood}.csv'), index=False)
    df_emission.to_csv(os.path.join(emission_dir, f'{half_life_solid_wood}.csv'), index=False)

In [19]:
# Initialize Forest Model
fm = initialize_forest_model()

found match for mask ('?', '?', '2401000', '?', '2401000')
found match for mask ('?', '?', '2401000', '?', '2401000')
found match for mask ('?', '?', '2401001', '?', '2401001')
found match for mask ('?', '?', '2401001', '?', '2401001')
found match for mask ('?', '?', '2401003', '?', '2401003')
found match for mask ('?', '?', '2401003', '?', '2401003')
found match for mask ('?', '?', '2401004', '?', '2401004')
found match for mask ('?', '?', '2401004', '?', '2401004')
found match for mask ('?', '?', '2401005', '?', '2401005')
found match for mask ('?', '?', '2401005', '?', '2401005')
found match for mask ('?', '?', '2401006', '?', '2401006')
found match for mask ('?', '?', '2401006', '?', '2401006')
found match for mask ('?', '?', '2402001', '?', '2402001')
found match for mask ('?', '?', '2402001', '?', '2402001')
found match for mask ('?', '?', '2402005', '?', '2402005')
found match for mask ('?', '?', '2402005', '?', '2402005')
found match for mask ('?', '?', '2402006', '?', '2402006

In [ ]:
# Main loop over half_life_solid_wood from 1 to iteration time
for half_life_solid_wood in range(1, iteration+1):
    
    # Define coefficient function
    
    def cmp_c_se(fm, path, expr, yname, half_life_solid_wood=half_life_solid_wood, half_life_paper=half_life_paper, proportion_solid_wood=proportion_solid_wood, util=util, displacement_factor_solid=displacement_factor_solid, credit=credit, mask=None):
        
        """
        Compile objective function coefficient for net system carbon emission indicators 
        (given ForestModel instance, leaf-to-root-node path, and expression to evaluate).
        """
        
        hwps_solid_pool = 0.
        hwps_paper_pool = 0.
        hwps_residue_pool = 0.
        hwps_solid_emission = 0.
        hwps_paper_emission = 0.
        hwps_residue_emission = 0.
        result = 0.
        
        # Calculate decay rates based on half-lives
        k_solid_wood = math.log(2) / half_life_solid_wood  # Decay rate for solid wood products (30-year half-life)
        k_paper = math.log(2) / half_life_paper  # Decay rate for paper (2-year half-life)
        
        # Define the allocation distribution
        proportion_paper = 1 - proportion_solid_wood
        
        # wood density (Kennedy, 1965)
        wood_density = 460

        # carbon content
        carbon_content = 0.5
        
        for t, n in enumerate(path, start=1):
            
            d = n.data()

            eco_emission = fm.inventory(t, 'net_emission', age=d['_age'], dtype_keys=[d['_dtk']])
            
            # Track the new product stock
            if fm.is_harvest(d['acode']):             
                # Calculate new product stock
                new_product_stock = fm.compile_product(t, expr, d['acode'], [d['dtk']], d['age'], coeff=False) * wood_density * carbon_content / 1000  # Convert kg to ton
            else:
                new_product_stock = 0.
            
            # Calculate product stock
            hwps_solid_pool = hwps_solid_pool * (1 - k_solid_wood)**10 + new_product_stock * util * proportion_solid_wood
            hwps_paper_pool = hwps_paper_pool * (1 - k_paper)**10 + new_product_stock * util * proportion_paper
            hwps_residue_pool = new_product_stock * (1.0 - util)

            # Calculate prodcut emission
            hwps_solid_emission = hwps_solid_pool * (1- (1 - k_solid_wood)**10)
            hwps_paper_emission = hwps_paper_pool * (1- (1 - k_paper)**10)
            hwps_residue_emission = hwps_residue_pool
            hwps_sum_emission = hwps_solid_emission + hwps_paper_emission + hwps_residue_emission
           
            # Calculate Substitution Effect
            substitution_effect_solid = new_product_stock * util * proportion_solid_wood * displacement_factor_solid * credit  # Emissions avoided by using HWPs
            substitution_effect_paper = 0.
            substitution_effect_residue = 0.
            substitution_effect_sum = substitution_effect_solid + substitution_effect_paper + substitution_effect_residue
            
            # Accumlate the total system carbon stock in each timestep
            result += (eco_emission + hwps_sum_emission - substitution_effect_sum)
        
        return result
        
    # Run the scenario and save results
    run_scenario_and_save(fm, half_life_solid_wood)